In [27]:
#Import libs
import time
import os, os.path
import random
import cv2
import matplotlib
import functools
import matplotlib.pyplot as plt
import tensorflow as tf
from Data_Generator import data_generator
import glob
import pandas as pd
import numpy as np

#Hyperparameters

batch_size = 50
Alpha = 0.5
margin = Alpha

# Generators
training_generator = data_generator('./test_example.txt',batch_size,True).generate()
test_generator = data_generator('./test_example.txt',batch_size,True).generate()

Initializing Generator...
Initializing Generator...


In [28]:
#No need for positive mask in Multimodal Triplet. Anchor and Positives will be fed through data inputs.'''
def _pairwise_distances(image_embeddings, text_embeddings, squared=False):
    """Compute the 2D matrix of distances between all the embeddings.
    Args:
        embeddings: tensor of shape (batch_size, embed_dim)
        squared: Boolean. If true, output is the pairwise squared euclidean distance matrix.
                 If false, output is the pairwise euclidean distance matrix.
    Returns:
        pairwise_distances: tensor of shape (batch_size, batch_size)
    """
    # Get the dot product between all embeddings
    # shape (batch_size, batch_size)
    dot_product11 = tf.matmul(image_embeddings, tf.transpose(image_embeddings))
    dot_product22 = tf.matmul(text_embeddings, tf.transpose(text_embeddings))
    dot_product12 = tf.matmul(image_embeddings, tf.transpose(text_embeddings))

    # Get squared L2 norm for each embedding. We can just take the diagonal of `dot_product`.
    # This also provides more numerical stability (the diagonal of the result will be exactly 0).
    # shape (batch_size,)
    square_norm11 = tf.diag_part(dot_product11)
    square_norm22 = tf.diag_part(dot_product22)

    # Compute the pairwise distance matrix as we have:
    # ||a - b||^2 = ||a||^2  - 2 <a, b> + ||b||^2
    # shape (batch_size, batch_size)
    distances = tf.expand_dims(square_norm11, 0) - 2.0 * dot_product12 + tf.expand_dims(square_norm22, 1)

    # Because of computation errors, some distances might be negative so we put everything >= 0.0
    distances = tf.maximum(distances, 0.0)

    if not squared:
        # Because the gradient of sqrt is infinite when distances == 0.0 (ex: on the diagonal)
        # we need to add a small epsilon where distances == 0.0
        mask = tf.to_float(tf.equal(distances, 0.0))
        distances = distances + mask * 1e-16

        distances = tf.sqrt(distances)

        # Correct the epsilon added: set the distances on the mask to be exactly 0.0
        distances = distances * (1.0 - mask)

    return distances
#No need for positive mask in Multimodal Triplet. Anchor and Positives will be fed through data inputs.

def _get_anchor_negative_triplet_mask(labels):
    """Return a 2D mask where mask[a, n] is True iff a and n have distinct labels.
    Args:
        labels: tf.int32 `Tensor` with shape [batch_size]
    Returns:
        mask: tf.bool `Tensor` with shape [batch_size, batch_size]
    """
    # Check if labels[i] != labels[k]
    # Uses broadcasting where the 1st argument has shape (1, batch_size) and the 2nd (batch_size, 1)
    labels_equal = tf.equal(tf.expand_dims(labels, 0), tf.expand_dims(labels, 1))

    mask = tf.logical_not(labels_equal)

    return mask


def _get_triplet_mask(labels):
    """Return a 3D mask where mask[a, p, n] is True iff the triplet (a, p, n) is valid.
    A triplet (i, j, k) is valid if:
        - i == j, k is distinct
        - i == j and labels[i] != labels[k]
    Args:
        labels: tf.int32 `Tensor` with shape [batch_size]
    """
    # Check that i == j and k is distinct
    indices_equal = tf.cast(tf.eye(tf.shape(labels)[0]), tf.bool)
    indices_not_equal = tf.logical_not(indices_equal)
    i_equal_j = tf.expand_dims(indices_equal, 2)
    i_not_equal_k = tf.expand_dims(indices_not_equal, 1)
    j_not_equal_k = tf.expand_dims(indices_not_equal, 0)

    valid_indices = tf.logical_and(tf.logical_and(i_equal_j, i_not_equal_k), j_not_equal_k)


    # Check if labels[i] == labels[j] and labels[i] != labels[k]
    label_equal = tf.equal(tf.expand_dims(labels, 0), tf.expand_dims(labels, 1))
    i_equal_j = tf.expand_dims(label_equal, 2)
    i_equal_k = tf.expand_dims(label_equal, 1)

    valid_labels = tf.logical_and(i_equal_j, tf.logical_not(i_equal_k))

    # Combine the two masks
    mask = tf.logical_and(valid_indices, valid_labels)

    return mask

#No need for triplet mask. Negative Mask, serves as triplet mask for both L_tii and L_itt.

def batch_all_triplet_loss(labels, image_embeddings, text_embeddings, margin, squared=False):
    """Build the triplet loss over a batch of embeddings.
    We generate all the valid triplets and average the loss over the positive ones.
    Args:
        labels: labels of the batch, of size (batch_size,)
        embeddings: tensor of shape (batch_size, embed_dim)
        margin: margin for triplet loss
        squared: Boolean. If true, output is the pairwise squared euclidean distance matrix.
                 If false, output is the pairwise euclidean distance matrix.
    Returns:
        triplet_loss: scalar tensor containing the triplet loss
    """
    
    # Get the pairwise distance matrix for L_itt
    pairwise_dist = _pairwise_distances(image_embeddings, text_embeddings)
    
    # Get the pairwise distance matrix for L_tii
    pairwise_dist_transpose = tf.transpose(pairwise_dist)

    # shape (batch_size, batch_size, 1)
    #Anchor Positive Dist is same for L_tii and L_itt
    #identity_mask = tf.eye(tf.shape(pairwise_dist))
    #anchor_positive_dist = tf.expand_dims(tf.multiply(identity_mask,pairwise_dist), 2)
    anchor_positive_dist = tf.expand_dims(pairwise_dist, 2)
    print(tf.shape(anchor_positive_dist))
    #assert anchor_positive_dist.shape[2] == 1, "{}".format(anchor_positive_dist.shape)
    
    # shape (batch_size, 1, batch_size)
    anchor_negative_dist = tf.expand_dims(pairwise_dist, 1) #L_itt
    anchor_negative_dist_transpose = tf.expand_dims(pairwise_dist_transpose, 1) #L_tii
    #assert anchor_negative_dist.shape[1] == 1, "{}".format(anchor_negative_dist.shape)
    #assert anchor_negative_dist_transpose.shape[1] == 1, "{}".format(anchor_negative_dist_transpose.shape)

    # Compute a 3D tensor of size (batch_size, batch_size, batch_size)
    # triplet_loss[i, j, k] will contain the triplet loss of anchor=i, positive=j, negative=k
    # Uses broadcasting where the 1st argument has shape (batch_size, batch_size, 1)
    # and the 2nd (batch_size, 1, batch_size)
    L_itt = anchor_positive_dist - anchor_negative_dist + margin
    L_tii = anchor_positive_dist - anchor_negative_dist_transpose + margin

    # Put to zero the invalid triplets
    # (where label(a) != label(p) or label(n) == label(a) or a == p)
    mask = _get_triplet_mask(labels)
    mask = tf.to_float(mask)
    L_itt = tf.multiply(mask, L_itt)
    L_tii = tf.multiply(mask, L_tii)

    # Remove negative losses (i.e. the easy triplets)
    L_itt = tf.maximum(L_itt, 0.0)
    L_tii = tf.maximum(L_tii, 0.0)
    

    # Count number of positive triplets (where triplet_loss > 0)
    valid_triplets_tii = tf.to_float(tf.greater(L_tii, 1e-16))
    valid_triplets_itt = tf.to_float(tf.greater(L_itt, 1e-16))
    
    num_positive_triplets_tii = tf.reduce_sum(valid_triplets_tii)
    num_positive_triplets_itt = tf.reduce_sum(valid_triplets_itt)
    
    num_valid_triplets = tf.reduce_sum(mask)
    fraction_positive_triplets = (num_positive_triplets_tii + num_positive_triplets_itt) / (2*num_valid_triplets + 1e-16)

    # Get final mean triplet loss over the positive valid triplets
    triplet_loss = (tf.reduce_sum(L_tii) + tf.reduce_sum(L_itt)) / (num_positive_triplets_tii + num_positive_triplets_itt + 1e-16)
    
    return triplet_loss, num_positive_triplets_tii, num_positive_triplets_itt

In [29]:
#############################################################################################################

#############################################################################################################

###################################### tensorflow network ###################################################

def image_net(image_dict, reuse, is_training):
    with tf.variable_scope('ConvNet', reuse=reuse):
        image_input= image_dict
        image_input = tf.reshape(image_input, shape=[-1,512,512,3])
        conv1 = tf.layers.conv2d(image_input, 32, 3, activation=tf.nn.relu)
        conv1 = tf.layers.average_pooling2d(conv1,2,4)
        conv2 = tf.layers.conv2d(conv1, 128, 3, activation = tf.nn.sigmoid)
        conv2 = tf. layers.average_pooling2d(conv2,2,4)
        conv3 = tf.layers.conv2d(conv2, 512, 3, activation = tf.nn.sigmoid)
        conv3 = tf.layers.average_pooling2d(conv3,2,4)
        flat = tf.contrib.layers.flatten(conv3)
        tf.summary.histogram('Image_flatten', flat)
        fc1 = tf.layers.dense(flat, 512, activation=None, name='fc1_dense')
        out = tf.layers.batch_normalization(fc1, training=is_training,name='batchnorm')
        tf.summary.histogram('Image_batch_norm_layer', out)
        out = tf.identity(out, name ='my_feature_embedding')
        tf.summary.histogram('Image_final_activation', out)
        
    return out 

def text_net (text_dict, reuse, is_training): 
    with tf.variable_scope('TextNet', reuse=reuse):
        text_input = text_dict
        fc1 = tf.layers.dense(text_input, 300, activation = tf.nn.sigmoid)
        fc2 = tf.layers.dense(fc1, 400, activation = tf.nn.sigmoid)
        fc3 = tf.layers.dense(fc2, 450, activation= tf.nn.sigmoid)
        fc4 = tf.layers.dense(fc3, 512, activation = None)
        tf.summary.histogram('Text_dense_4', fc4)
        out = tf.layers.batch_normalization(fc4, training=is_training,name='text_batchnorm')
        tf.summary.histogram('Text_batch_norm_layer', out)
        out = tf.identity(out, name='my_text_embedding')
        tf.summary.histogram('Text_final_activation', out)
    return out
 

In [30]:
   
###################################################################################################################
tf.reset_default_graph()
with tf.name_scope("inputs"):
    reuse = False
    learning_rate = 0.001
    image_dict = tf.placeholder(tf.float32, shape=(batch_size,512,512,3))
    text_dict = tf.placeholder(tf.float32, shape = (batch_size,200))
    labels = tf.placeholder(tf.float32, shape=(batch_size))
    is_training = tf.placeholder_with_default(False, shape=[], name='training')

In [31]:
####################################################################################################################

image_embeddings = image_net(image_dict, reuse, is_training)
text_embeddings = text_net(text_dict, reuse, is_training)
image_embedding_mean_norm = tf.reduce_mean(tf.norm(image_embeddings, axis=1))
tf.summary.scalar("embedding_image_mean_norm", image_embedding_mean_norm)
text_embedding_mean_norm = tf.reduce_mean(tf.norm(text_embeddings, axis=1))
tf.summary.scalar("embedding_text_mean_norm", text_embedding_mean_norm)
# update operation to update batch-norm variables
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    my_loss, num_tii, num_itt = batch_all_triplet_loss(labels, image_embeddings, text_embeddings, margin, squared=False)
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, name='Adam-op')
    minimizer = optimizer.minimize(my_loss)
    
    
##########################################################################################################################

with tf.name_scope("Init"):
    merged = tf.summary.merge_all()
    init = tf.global_variables_initializer()
    graph = tf.get_default_graph()
    saver = tf.train.Saver()

Tensor("Shape:0", shape=(3,), dtype=int32)


In [32]:
    
##########################################################################################################################
################################################# TRAINING SESSION ######################################################

In [34]:
with tf.Session() as sess:
    print("Initializing Session")
    sess.run(init)
    print("Init Done")
    train_writer = tf.summary.FileWriter('./Mera_train/train',sess.graph)
    test_writer = tf.summary.FileWriter('./Mera_test/test',sess.graph)
    global_step = 0
    count = 1
    for [images, texts], labels_input in training_generator:
        if count < 10:
            print("Running Batch Number", count)
        feed_dict_batch = {image_dict: images, text_dict: texts, labels: labels_input,is_training: True}
        loss_val, _num_tii, _num_itt, minimized = sess.run([my_loss, num_tii, num_itt, minimizer], feed_dict=feed_dict_batch)
        if count%20 == 0 or count < 10:
            print(count, " -- Loss Val: ", loss_val, " -- Trip_TII: ", _num_tii, " -- Trip_ITT: ", _num_itt)
            summary, _ = sess.run([merged,minimizer], feed_dict=feed_dict_batch)
            train_writer.add_summary(summary, count)
        if count%50 == 0:
            save_path = saver.save(sess, "./models/model"+str(count)+".ckpt")
            print("Model saved in path: %s" % save_path)
        if count%100 == 0:
            break
        count+=1

Initializing Session
Init Done
Running Batch Number 1
1  -- Loss Val:  0.69343215  -- Trip_TII:  2450.0  -- Trip_ITT:  1758.0
Running Batch Number 2
2  -- Loss Val:  1.8151816  -- Trip_TII:  1734.0  -- Trip_ITT:  1252.0
Running Batch Number 3
3  -- Loss Val:  1.3507627  -- Trip_TII:  629.0  -- Trip_ITT:  934.0
Running Batch Number 4
4  -- Loss Val:  0.85602367  -- Trip_TII:  309.0  -- Trip_ITT:  515.0
Running Batch Number 5
5  -- Loss Val:  0.6237419  -- Trip_TII:  79.0  -- Trip_ITT:  340.0
Running Batch Number 6
6  -- Loss Val:  0.57980317  -- Trip_TII:  130.0  -- Trip_ITT:  235.0
Running Batch Number 7
7  -- Loss Val:  0.5386916  -- Trip_TII:  71.0  -- Trip_ITT:  85.0
Running Batch Number 8
8  -- Loss Val:  0.5319695  -- Trip_TII:  69.0  -- Trip_ITT:  83.0
Running Batch Number 9
9  -- Loss Val:  0.5597418  -- Trip_TII:  13.0  -- Trip_ITT:  15.0
20  -- Loss Val:  0.27966687  -- Trip_TII:  15.0  -- Trip_ITT:  15.0
40  -- Loss Val:  1.8118562  -- Trip_TII:  3.0  -- Trip_ITT:  10.0
Model

In [35]:
load_image_data_flag = 1
load_sig_data=0
batch_size = 1
count=500
margin=0.5

In [36]:
image_paths = []
text_paths = []
labels = []
image_data=[]
with open('./test_example.txt', 'r') as datafile:
    for line in datafile:
        [image_path, text_path, label] = line.split(" ")
        image_paths.append(image_path)
        text_paths.append(text_path)
        labels.append(label)
for image in image_paths:
            image_data.append(cv2.imread(image))


In [56]:
image_data = np.array(image_data)

labels = np.array(labels)

In [61]:
test_embeddings=[]
image_data1=image_data[:50]

In [82]:
labels_data1=labels[:50]

In [62]:
image_data1.shape

(50, 512, 512, 3)

In [64]:
with tf.Session() as sess:
        print("Initializing Session")
        sess.run(init)
        saver = tf.train.import_meta_graph('./models/model50.ckpt.meta')  # Put the meta file here
        saver.restore(sess, "./models/model50.ckpt")
#        saver.restore(sess, "models/run_6/checkpoint")
        print("Model Restored")
        print("Init Done")
        #test_writer = tf.summary.FileWriter('./Graph_run6/test',sess.graph)
        image_embeddings = graph.get_tensor_by_name("ConvNet/my_feature_embedding:0")
        image_dict = graph.get_tensor_by_name("inputs/Placeholder:0") #same as above
        
        # Running for the Total_size/batch_size times
        count=0
        feed_dict_batch= {image_dict:image_data1 ,is_training:True}
        test_embedding = sess.run(image_embeddings, feed_dict=feed_dict_batch)
        test_embeddings.append(test_embedding)
        print("done")

Initializing Session
INFO:tensorflow:Restoring parameters from ./models/model50.ckpt
Model Restored
Init Done
done


In [67]:
test_embeddings_np = np.array(test_embeddings)

In [76]:
test_embeddings_np

array([[[ 0.12944031,  0.00280762, -0.08598328, ...,  0.03320312,
          0.09320068, -0.2600708 ],
        [-0.08644867, -0.08703613,  0.01959229, ..., -0.09265137,
          0.11755371, -0.38360596],
        [-0.28652954, -0.01855469,  0.31063843, ...,  0.10192871,
         -0.27368164, -0.12030029],
        ...,
        [ 0.36410522, -0.62109375,  0.05033875, ..., -0.02185059,
         -0.38079834, -0.2781372 ],
        [ 0.38237762, -0.24707031, -0.22154236, ..., -0.23449707,
         -0.26446533,  0.19818115],
        [-0.48609924,  0.60424805, -0.82366943, ..., -0.07299805,
          0.7598877 , -0.06854248]]], dtype=float32)

In [70]:
test_emb_mean = np.mean(test_embeddings_np,axis=2)

In [73]:
test_emb_mean

array([[-0.00702539, -0.01667348, -0.01516603, -0.00732607, -0.01955907,
        -0.02634917, -0.02073777,  0.03263785,  0.02569321,  0.02530759,
        -0.02510426, -0.02489255, -0.01473117,  0.01851849, -0.01894451,
        -0.01249103,  0.00777126,  0.01938887,  0.01559644, -0.00565375,
        -0.00407266,  0.00350026,  0.03871414, -0.01240822,  0.01022159,
         0.01005187,  0.03707859, -0.02371478,  0.00238076, -0.01427337,
        -0.01440587, -0.02118009,  0.01575859, -0.00063899, -0.01007861,
         0.01574463,  0.00449814,  0.01721081,  0.00929052,  0.01651668,
         0.01731256, -0.00186087, -0.00613387,  0.00584374,  0.01233236,
        -0.0244056 , -0.00563774, -0.03469304,  0.01016186, -0.00342729]],
      dtype=float32)

In [74]:
np.unique(test_emb_mean)

array([-0.03469304, -0.02634917, -0.02510426, -0.02489255, -0.0244056 ,
       -0.02371478, -0.02118009, -0.02073777, -0.01955907, -0.01894451,
       -0.01667348, -0.01516603, -0.01473117, -0.01440587, -0.01427337,
       -0.01249103, -0.01240822, -0.01007861, -0.00732607, -0.00702539,
       -0.00613387, -0.00565375, -0.00563774, -0.00407266, -0.00342729,
       -0.00186087, -0.00063899,  0.00238076,  0.00350026,  0.00449814,
        0.00584374,  0.00777126,  0.00929052,  0.01005187,  0.01016186,
        0.01022159,  0.01233236,  0.01559644,  0.01574463,  0.01575859,
        0.01651668,  0.01721081,  0.01731256,  0.01851849,  0.01938887,
        0.02530759,  0.02569321,  0.03263785,  0.03707859,  0.03871414],
      dtype=float32)

In [80]:
 test_embeddings_final=test_embeddings_np.reshape((50,512))
    

In [81]:
test_embeddings_final.shape

(50, 512)

In [83]:
labels_data1.shape

(50,)

In [91]:
!conda install --yes faiss-cpu -c pytorch

Solving environment: done

## Package Plan ##

  environment location: /home/ayush/anaconda3

  added / updated specs: 
    - faiss-cpu


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    gstreamer-1.14.0           |       hb453b48_1         3.8 MB
    fribidi-1.0.5              |       h7b6447c_0         112 KB
    tk-8.6.8                   |       hbc83047_0         3.1 MB
    cryptography-2.6.1         |   py36h1ba5d50_0         609 KB
    libcurl-7.64.0             |       h20c2e04_2         600 KB
    krb5-1.16.1                |       h173b8e3_7         1.4 MB
    pcre-8.43                  |       he6710b0_0         260 KB
    fontconfig-2.13.0          |       h9420a91_0         291 KB
    pango-1.42.4               |       h049681c_0         528 KB
    python-3.6.8               |       h0371630_0        34.4 MB
    cairo-1.14.12              |       h8948797_3         1.3 MB
   

pillow 5.4.1############################################################ | 100% 
pycurl 7.43.0.2######################################################### | 100% 
certifi 2019.3.9######################################################## | 100% 
libgcc-ng 8.2.0######################################################### | 100% 
libuuid 1.0.3########################################################### | 100% 
libxcb 1.13############################################################# | 100% 
gst-plugins-base 1.14.0################################################# | 100% 
ca-certificates 2019.1.23############################################### | 100% 
openssl 1.1.1b########################################################## | 100% 
libxml2 2.9.9########################################################### | 100% 
curl 7.64.0############################################################# | 100% 
harfbuzz 1.8.8########################################################## | 100% 
sqlite 3.27.2###############

In [93]:
import faiss
import sys
d=512
nb = test_embeddings_final.shape[0]
nq = labels_data1.shape[0]
index = faiss.IndexFlatL2(d)
print(index.is_trained)
sys.stdout.flush()
index.add(test_embeddings_final)                  # add vectors to the index
print(index.ntotal)
sys.stdout.flush()
k = 1
print("Doing Sanity Check")
sys.stdout.flush()
D, I = index.search(test_embeddings_final, k) # sanity check
print(I)
print(D)

True
50
Doing Sanity Check
[[ 0]
 [ 1]
 [ 2]
 [ 3]
 [ 4]
 [ 5]
 [ 6]
 [ 7]
 [ 8]
 [ 9]
 [10]
 [11]
 [12]
 [13]
 [14]
 [15]
 [16]
 [17]
 [18]
 [19]
 [20]
 [21]
 [22]
 [23]
 [24]
 [25]
 [26]
 [27]
 [28]
 [29]
 [30]
 [31]
 [32]
 [33]
 [34]
 [35]
 [36]
 [37]
 [38]
 [39]
 [40]
 [41]
 [42]
 [43]
 [44]
 [45]
 [46]
 [47]
 [48]
 [49]]
[[0.0000000e+00]
 [0.0000000e+00]
 [1.5258789e-05]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [1.5258789e-05]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [7.6293945e-06]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [7.6293945e-06]
 [0.0000000e+00]
 [0.0000000e+00]
 [1.5258789e-05]
 [0.0000000e+00]
 [7.6293945e-06]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.0000000e+00]
 [0.00000